In [25]:
import json
from collections import defaultdict
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [26]:
df = pd.read_csv("../Data/Interim/some_cmd_v3_03062022_low_split.csv", sep=",")
df = df.loc[:,["full_text", "text1", "text2", "text3", "subset"]]
print(df.shape)
# добавляем кома
df_onecmd = pd.read_csv("../Data/Interim/one_cmd_v3_03062022_low_split.csv", sep=",")
df_onecmd = df_onecmd.sample(5000, random_state=42)
df_onecmd = df_onecmd.loc[df_onecmd["subset"].isin(["train", "valid"]), ["x", "subset"]]
df_onecmd["full_text"] = df_onecmd["x"]
df_onecmd["text1"] = df_onecmd["x"]
df_onecmd["text2"] = np.empty_like(df_onecmd["x"], dtype=np.float32)
df_onecmd["text3"] = np.empty_like(df_onecmd["x"], dtype=np.float32)

df = pd.concat([df, df_onecmd])
df.index= range(len(df))

(16253, 5)


In [3]:
nerInput_train = {
    "sentence_id": [],
    "words": [],
    "labels": []
}
nerInput_valid = {
    "sentence_id": [],
    "words": [],
    "labels": []
}
nerInput = None
for ir, row in df.iterrows():
    if row["subset"]=="train":
        nerInput = nerInput_train
    if row["subset"]=="valid":
        nerInput = nerInput_valid

    start_1 = row["full_text"].find(row["text1"])
    assert start_1 >= 0
    start_2 = None
    if type(row['text2']) != float:
        start_2 = row["full_text"].find(row["text2"], start_1 + len(row["text1"]))    
        assert start_2 >= start_1
    start_3 = None
    if type(row['text3']) != float:
        start_3 = row["full_text"].find(row["text3"], start_2 + len(row["text2"]))
        assert start_3 >= start_2
    lastWordPos = 0
    for word in row["full_text"].split():
        nerInput["sentence_id"].append(ir)
        nerInput["words"].append(word)
        wordPos = row["full_text"].find(word, lastWordPos)
        assert wordPos!=-1
        lastWordPos = wordPos + len(word)
        if (wordPos >= start_1) and (wordPos <= (start_1+len(row["text1"].strip()))):
            if lastWordPos == (start_1+len(row["text1"].strip())):
                nerInput["labels"].append("[SEP]")
            else:
                nerInput["labels"].append("[CMD]")
        elif (start_2 is not None) and (wordPos >= start_2) and (wordPos <= (start_2+len(row["text2"].strip()))):
            if lastWordPos == (start_2+len(row["text2"].strip())):
                nerInput["labels"].append("[SEP]")
            else:
                nerInput["labels"].append("[CMD]")
        elif (start_3 is not None) and (wordPos >= start_3) and (wordPos <= (start_3+len(row["text3"].strip()))):
            if lastWordPos == (start_3+len(row["text3"].strip())):
                nerInput["labels"].append("[SEP]")
            else:
                nerInput["labels"].append("[CMD]")
        else:
            nerInput["labels"].append("O")
train_df = pd.DataFrame(nerInput_train)
valid_df = pd.DataFrame(nerInput_valid)

In [4]:
for ir, row in df.iterrows():
    cmd_count = 1
    if type(row["text2"])==str:
        cmd_count+=1
    if type(row["text3"])==str:
        cmd_count+=1
    if ir in train_df["sentence_id"].values:
        assert train_df[train_df["sentence_id"]==ir]["labels"].value_counts()["[SEP]"]==cmd_count
    else:
        assert valid_df[valid_df["sentence_id"]==ir]["labels"].value_counts()["[SEP]"]==cmd_count

In [5]:
for ir, row in df.iterrows():
    cmd_count = 1
    if type(row["text2"])==str:
        cmd_count+=1
    if type(row["text3"])==str:
        cmd_count+=1
    ner_commands = [[],[],[]]
    current_command = 0
    if ir in train_df["sentence_id"].values:
        input_df = train_df
    else:
        input_df = valid_df
    for ir_2, row_2 in input_df[input_df["sentence_id"]==ir].iterrows():
        if row_2["labels"]=="O":
            continue
        ner_commands[current_command].append(row_2["words"])
        if row_2["labels"]=="[SEP]":
            current_command += 1
    assert row["text1"].strip() == " ".join(ner_commands[0])
    if cmd_count>1:
        assert row["text2"].strip() == " ".join(ner_commands[1])
    if cmd_count>2:
        assert row["text3"].strip() == " ".join(ner_commands[2])

In [6]:
from simpletransformers.ner import NERModel, NERArgs
model_args = NERArgs()
model_args.evaluate_during_training = True
model_args.num_train_epochs = 10
model_args.output_dir = "/s/ls4/users/grartem/RL_robots/CommandClassifier/models/TokenCLS_sepPredictor_ipynb_Copy1_data3v2"

model = NERModel(
    "bert", 
    "/s/ls4/users/grartem/HuggingFace/models/rubert-tiny2",
    #"/s/ls4/users/grartem/RL_robots/CommandClassifier/models/TokenCLS_sepPredictor_ipynb/checkpoint-20664-epoch-9/",
    args=model_args,
    labels=["[CMD]", "[SEP]","O"]
)

Some weights of the model checkpoint at /s/ls4/users/grartem/HuggingFace/models/rubert-tiny2 were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassific

In [7]:
# Train the model
model.train_model(train_df, eval_data=valid_df)

  0%|          | 0/29 [00:00<?, ?it/s]

/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/2264 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

...

/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: [SEP] seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: [CMD] seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


  0%|          | 0/7 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/380 [00:00<?, ?it/s]

/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: [SEP] seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: [CMD] seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


(22640,
 defaultdict(list,
             {'global_step': [2000,
               2264,
               4000,
               4528,
               6000,
               6792,
               8000,
               9056,
               10000,
               11320,
               12000,
               13584,
               14000,
               15848,
               16000,
               18000,
               18112,
               20000,
               20376,
               22000,
               22640],
              'train_loss': [0.03387736901640892,
               0.0007173155900090933,
               0.00029832852305844426,
               0.0001819054305087775,
               7.245669257827103e-05,
               5.845365740242414e-05,
               5.360653449315578e-05,
               4.220008850097656e-05,
               2.2140204237075523e-05,
               1.5078364413056988e-05,
               1.055002212524414e-05,
               1.0685920642572455e-05,
               1.58312177518382

In [8]:
# Evaluate the model
result, model_outputs, preds_list = model.eval_model(valid_df)

  0%|          | 0/7 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/380 [00:00<?, ?it/s]

/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: [SEP] seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: [CMD] seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


In [20]:
predictions, raw_outputs = model.predict(df[df["subset"]=="valid"]["full_text"])

  0%|          | 0/7 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/380 [00:00<?, ?it/s]

# оценка

In [21]:
from sklearn.metrics import classification_report
flat_token_predictions = [list(x.values())[0] for sent in predictions for x in sent]
flat_token_true = valid_df["labels"].values
print(classification_report(flat_token_true, flat_token_predictions))

              precision    recall  f1-score   support

           O       1.00      1.00      1.00      2959
       [CMD]       1.00      1.00      1.00     23669
       [SEP]       1.00      1.00      1.00      5473

    accuracy                           1.00     32101
   macro avg       1.00      1.00      1.00     32101
weighted avg       1.00      1.00      1.00     32101



In [22]:
i, errors = 0, 0
for ir, row in df[df["subset"]=="valid"].iterrows():
    text1, text2, text3 = [], [], []
    cmd_i = 0
    for token in predictions[i]:
        token, cls = list(token.items())[0]
        if cls=="O":
            continue
        if cmd_i==0:
            text1.append(token)
        elif cmd_i==1:
            text2.append(token)
        elif cmd_i==2:
            text3.append(token)
        if cls=="[SEP]":
            cmd_i += 1
    text1 = " ".join(text1)
    text2 = " ".join(text2)
    text3 = " ".join(text3)
    
    try:
        assert row["text1"].strip() in text1
        assert (text2=="" and type(row["text2"]) is float) or row["text2"].strip() in text2
        assert (text3=="" and type(row["text3"]) is float) or row["text3"].strip() in text3
        #assert (text3=="" and np.isnan(row["text3"])) or text3 == row["text3"].strip()
    except AssertionError:
        print("full_text:", row["full_text"])
        print("text1:", row["text1"], "predict:", text1)
        print("text2:", row["text2"], "predict:", text2)
        print("text3:", row["text3"], "predict:", text3)
        print()
        errors += 1
    i+=1
print("Errors", errors)

full_text: обнаружь железного коня спереди от лесополосы находящуюся сзади от булыжника по прошествии времени справа от лесополосы находится светильник который приказывается рассмотреть
text1: обнаружь железного коня спереди от лесополосы находящуюся сзади от булыжника predict: обнаружь железного коня
text2: справа от лесополосы находится светильник который приказывается рассмотреть predict: спереди от лесополосы находящуюся сзади от булыжника
text3: nan predict: справа от лесополосы находится светильник который приказывается рассмотреть

full_text: сделай поворот до канавы передохни с течением времени приспевай назад
text1: сделай поворот до канавы predict: сделай поворот до канавы передохни
text2: передохни predict: приспевай назад
text3: приспевай назад predict: 

full_text: начни поворачивать 343° немного погодя стоять правее дерева располагается тачка к которой приказывается плыть
text1: начни поворачивать 343° predict: начни поворачивать 343°
text2: стоять predict: стоять правее 

# проверим, как она хорошо предсказывает фразы из 1 команды, если не видела их

In [23]:
df = pd.read_csv("../Data/Interim/one_cmd_v3_03062022_low_split.csv", sep=",")
predictions, raw_outputs = model.predict(df["x"])

  0%|          | 0/29 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/7136 [00:00<?, ?it/s]

In [24]:
errors_count = 0
for phrase in predictions:
    classes = [list(x.values())[0] for x in phrase]
    if classes[-1]=="[SEP]" and len([x for x in classes if x=="[SEP]"])==1:
        continue
    else:
        errors_count+=1
errors_count, len(predictions)

(295, 57085)